In [1]:
# !pip install requests pandas matplotlib

import requests
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

In [15]:
BASE_URL_OIL = "https://iss.moex.com/iss/history/engines/futures/markets/forts/securities/BR.json"

def load_brent_futures_history(from_date: str, till_date: str) -> pd.DataFrame:
    """
    Загружает историю фьючерса на нефть Brent (тикер BR) с Мосбиржи (рынок FORTS)
    за указанный диапазон дат (включительно) через ISS API.

    Параметры:
        from_date: начало периода, строка 'YYYY-MM-DD'
        till_date: конец периода, строка 'YYYY-MM-DD'

    Возвращает:
        pandas.DataFrame с колонками TRADEDATE, OPEN, HIGH, LOW, CLOSE, VOLUME и т.д.
    """
    all_rows = []
    start = 0
    step = 100  # стандартный шаг ISS

    while True:
        params = {
            "from": from_date,
            "till": till_date,
            "start": start
        }

        resp = requests.get(BASE_URL_OIL, params=params)
        resp.raise_for_status()
        data = resp.json()

        columns = data["history"]["columns"]
        rows = data["history"]["data"]

        if not rows:
            break

        df_page = pd.DataFrame(rows, columns=columns)
        all_rows.append(df_page)

        start += step

    if not all_rows:
        return pd.DataFrame()

    df = pd.concat(all_rows, ignore_index=True)

    # приведение даты и сортировка
    if "TRADEDATE" in df.columns:
        df["TRADEDATE"] = pd.to_datetime(df["TRADEDATE"])
        df = df.sort_values("TRADEDATE").reset_index(drop=True)

    return df

In [13]:
def get_brent_futures(
    from_date: str,
    till_date: str
    ,columns=("TRADEDATE", "OPEN", "HIGH", "LOW", "CLOSE", "VOLUME")
) -> pd.DataFrame:
    """
    Удобная обёртка над load_brent_futures_history.
    Возвращает только выбранные колонки.
    """
    df = load_brent_futures_history(from_date, till_date)
    if columns:
        cols = [c for c in columns if c in df.columns]
        df = df[cols]
    return df

In [16]:
# пример: Brent фьючерсы за 2024 год
df_brent_2024 = load_brent_futures_history("2024-01-01", "2024-12-31")
df_brent_2024.head()

""


In [5]:
plt.figure(figsize=(12, 5))
plt.plot(df_brent_2024["TRADEDATE"], df_brent_2024["CLOSE"])
plt.title("Фьючерс на нефть Brent (BR, Мосбиржа) – цена закрытия")
plt.xlabel("Дата")
plt.ylabel("Цена (руб. за контрактную единицу)")
plt.grid(True)
plt.tight_layout()
plt.show()

KeyError: 'TRADEDATE'

<Figure size 1200x500 with 0 Axes>

In [ ]:
from pathlib import Path

def save_oil_to_files(df: pd.DataFrame, base_name: str = "brent_futures", folder: str = "data"):
    """
    Сохраняет DataFrame с фьючерсом Brent в CSV и Excel.

    Параметры:
        df        : DataFrame с данными
        base_name : базовое имя файла без расширения
        folder    : папка для сохранения
    """
    folder_path = Path(folder)
    folder_path.mkdir(parents=True, exist_ok=True)

    csv_path = folder_path / f"{base_name}.csv"
    xlsx_path = folder_path / f"{base_name}.xlsx"

    df.to_csv(csv_path, index=False, encoding="utf-8")
    df.to_excel(xlsx_path, index=False)

    print(f"CSV сохранён в:   {csv_path}")
    print(f"Excel сохранён в: {xlsx_path}")


# пример сохранения:
save_oil_to_files(df_brent_2024, base_name="brent_2024", folder="data")